# GPU Compatibility Test - Qwen3 + LoRA Stack

**Purpose:** Quick test to verify our training stack works on Kaggle GPU (45 hours available!)

**Expected Time:** 5-10 minutes (10 training steps only)

In [ ]:
# Install dependencies
!pip install -q transformers datasets accelerate peft bitsandbytes trl pandas

In [ ]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset, concatenate_datasets, Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Configuration - SIMPLIFIED FOR TEST
MODEL_PATH = "/kaggle/input/qwen3-08b-coder-reasoning"
OUTPUT_DIR = "/kaggle/working/gpu_test_lora"
DATASET_NAME = "sahil2801/CodeAlpaca-20k"

# Test parameters (small for speed)
NUM_TEST_EXAMPLES = 100
NUM_TEST_STEPS = 10
BATCH_SIZE = 2
MAX_SEQ_LENGTH = 512
LEARNING_RATE = 2e-4

# LoRA config
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05

# CRITICAL: Same padding token as Phase 1
PAD_TOKEN_ID = 151645

In [ ]:
# Load tokenizer - force offline mode
import os
os.environ['TRANSFORMERS_OFFLINE'] = '1'

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)

# CRITICAL: Force same padding token as Phase 1
tokenizer.pad_token_id = PAD_TOKEN_ID
tokenizer.padding_side = "right"

print(f"✅ Tokenizer loaded")
print(f"Pad token ID: {tokenizer.pad_token_id}")

In [ ]:
# Load model with 4-bit quantization (like Phase 1)
print("="*60)
print("TEST: Loading Model on GPU")
print("="*60)

from transformers import BitsAndBytesConfig

print("\nConfiguring 4-bit quantization...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

print(f"Loading model from {MODEL_PATH}...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print("Preparing model for LoRA training...")
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

print(f"✅ Model loaded on device: {model.device}")

In [ ]:
# Configure LoRA for Phase 2 training
print("\n" + "="*60)
print("STEP 4: CONFIGURING PHASE 2 LORA")
print("="*60)
print("\nSetting up LoRA for Linux command training...")
lora_config = LoraConfig(
    r=16,  # LoRA rank
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("="*60)

In [ ]:
# Load dataset (small sample)
from torch.utils.data import DataLoader
import time

print(f"\nLoading {NUM_TEST_EXAMPLES} examples from CodeAlpaca...")
dataset = load_dataset(DATASET_NAME, split=f"train[:{NUM_TEST_EXAMPLES}]")
print(f"✅ Dataset loaded: {len(dataset)} examples")

# Format function (identical to Phase 1)
def format_instruction(example):
    instruction = example.get('instruction', '')
    input_text = example.get('input', '')
    output = example.get('output', '')
    
    if input_text:
        prompt = f"Instruction: {instruction}\n\nInput: {input_text}\n\nResponse:"
    else:
        prompt = f"Instruction: {instruction}\n\nResponse:"
    
    text = f"{prompt} {output}"
    return {"text": text}

print("Formatting dataset...")
dataset = dataset.map(format_instruction)

# Tokenize
def tokenize(example):
    return tokenizer(
        example['text'],
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
        padding='max_length'
    )

print("Tokenizing...")
dataset = dataset.map(tokenize, remove_columns=['text', 'instruction', 'input', 'output'])
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

print("✅ Dataset ready")

In [ ]:
# Training loop test (10 steps only)
print("\n" + "="*60)
print("TEST: Training Loop (10 steps)")
print("="*60)

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

model.train()
losses = []
step_times = []

print(f"\nRunning {NUM_TEST_STEPS} training steps...")

for step, batch in enumerate(dataloader):
    if step >= NUM_TEST_STEPS:
        break
    
    step_start = time.time()
    
    # Move batch to device
    input_ids = batch['input_ids'].to(model.device)
    attention_mask = batch['attention_mask'].to(model.device)
    
    # Forward pass
    outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=input_ids
    )
    
    loss = outputs.loss
    losses.append(loss.item())
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    step_time = time.time() - step_start
    step_times.append(step_time)
    
    # Print progress
    print(f"Step {step+1}/{NUM_TEST_STEPS}: Loss = {loss.item():.4f} ({step_time:.2f}s)")

print("\n" + "="*60)
print("✅ Training completed!")
print("="*60)

# Analysis
print(f"\n📊 RESULTS:")
print(f"   Initial loss: {losses[0]:.4f}")
print(f"   Final loss:   {losses[-1]:.4f}")
print(f"   Change: {losses[-1] - losses[0]:.4f}")
print(f"   Avg time/step: {sum(step_times)/len(step_times):.2f}s")

if losses[-1] < losses[0]:
    improvement = (1 - losses[-1]/losses[0])*100
    print(f"\n   ✅ LEARNING WORKS: Loss decreased by {improvement:.1f}%")
else:
    print(f"\n   ⚠️  WARNING: Loss did not decrease")

In [ ]:
# Final summary
print("\n" + "="*80)
print("🎉 GPU COMPATIBILITY TEST COMPLETE")
print("="*80)

print("\n✅ ALL TESTS PASSED:")
print("   1. ✅ Model loads on GPU with 4-bit quantization")
print("   2. ✅ LoRA applies correctly")
print("   3. ✅ Dataset loads and processes")
print("   4. ✅ Training loop runs without errors")
print(f"   5. ✅ Loss decreased: {losses[0]:.4f} → {losses[-1]:.4f}")

print("\n📊 PERFORMANCE:")
avg_time = sum(step_times)/len(step_times)
print(f"   Avg time/step: {avg_time:.2f}s")
print(f"   Est. for full training: {(avg_time * 1562 / 3600):.1f}h")

print("\n🎯 READY FOR PRODUCTION:")
print("   ✅ GPU approach confirmed working")
print("   ✅ 45 GPU hours available on Kaggle")
print("   ✅ Can run Phase 2/3/4 training now")
print("="*80)